In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from collections import defaultdict

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../data/processed/train-normalized.csv.gz', skipinitialspace = True)

In [3]:
data.head()

,ncodpers,sexo,age,ind_nuevo,antiguedad,indrel,indresi,indext,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,indrel_1mes_5.0,indrel_1mes_6.0,indrel_1mes_1,indrel_1mes_1.0,indrel_1mes_2,indrel_1mes_2.0,indrel_1mes_3,indrel_1mes_3.0,indrel_1mes_4,indrel_1mes_4.0,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_N,tiprel_1mes_P,tiprel_1mes_R,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO
0,15889,1.0,0.333333,0.0,0.999999,1.0,1.0,0.0,1.0,0.011246,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,15890,1.0,0.376543,0.0,1.000000,1.0,1.0,0.0,1.0,0.002432,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,15891,0.0,0.351852,0.0,0.999990,0.0,1.0,0.0,0.0,0.003002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,15892,0.0,0.370370,0.0,1.000000,1.0,1.0,0.0,1.0,0.014857,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,15893,1.0,0.376543,0.0,1.000000,1.0,1.0,0.0,1.0,0.014857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
ids = data['ncodpers']
data.drop(['ncodpers'], axis=1, inplace=True)

In [5]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [6]:
for product in products:
        print(product)
        y_train = data[product]
        x_train = data.drop(product, 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[product] = clf
        model_preds[product] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))

ind_ahor_fin_ult1
0.857713999913
ind_aval_fin_ult1
0.987374360647
ind_cco_fin_ult1
0.850874194264
ind_cder_fin_ult1
0.921103609592
ind_cno_fin_ult1
0.992213937296
ind_ctju_fin_ult1
0.999577451741
ind_ctma_fin_ult1
0.862239329526
ind_ctop_fin_ult1
0.844884718116
ind_ctpp_fin_ult1
0.850952307017
ind_deco_fin_ult1
0.944938295856
ind_deme_fin_ult1
0.93965386605
ind_dela_fin_ult1
0.949304540515
ind_ecue_fin_ult1
0.896188614588
ind_fond_fin_ult1
0.934960754392
ind_hip_fin_ult1
0.957133750829
ind_plan_fin_ult1
0.930006475827
ind_pres_fin_ult1
0.94262483318
ind_reca_fin_ult1
0.896530859786
ind_tjcr_fin_ult1
0.942004445412
ind_valo_fin_ult1
0.91915600572
ind_viv_fin_ult1
0.866301521922
ind_nomina_ult1
0.999427664556
ind_nom_pens_ult1
0.998975709029
ind_recibo_ult1
0.939323262142


In [7]:
already_active = defaultdict(list)

for i in range(len(data)):  
    id_ = ids.at[i]
    active_products = []
    
    for p in products:
        if data.at[i, p] == 1.0:
            active_products.append(p)
    
    already_active[id_] = active_products

In [8]:
test = pd.read_csv('../data/raw/test.csv.zip', skipinitialspace = True, usecols=['ncodpers'])

In [9]:
most_popular = ['ind_cco_fin_ult1', 'ind_recibo_ult1', 'ind_ctop_fin_ult1', 'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1']

train_preds = {}

for id_ in test.ncodpers.values:
    preds_probas = id_preds[id_]
    active = already_active[id_]    
    predicted = []
    
    for prod_proba_idx in np.argsort(preds_probas)[::-1]:
        if products[prod_proba_idx] not in active and len(predicted) < 7:
            predicted.append(products[prod_proba_idx])
    
    train_preds[id_] = most_popular if len(predicted) == 0 else predicted

In [10]:
for ncodper in test.ncodpers.values:        
    train_preds[ncodper] = ' '.join(train_preds[ncodper])

In [11]:
predictions = pd.DataFrame(list(train_preds.items()), columns=['ncodpers', 'added_products'])
predictions.head()

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_ctop_fin_ult1 ind_ecue_fin...
1,15890,ind_valo_fin_ult1 ind_reca_fin_ult1 ind_dela_f...
2,15892,ind_plan_fin_ult1 ind_ctop_fin_ult1 ind_ctpp_f...
3,15893,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_ecue_fi...
4,15894,ind_cno_fin_ult1 ind_plan_fin_ult1 ind_ctpp_fi...


In [12]:
predictions.to_csv('../data/predicted.csv.gz', index=False, compression='gzip')